**1.** Wczytanie danych

In [ ]:
COUNTIES = '/content/drive/MyDrive/DataScience/data/counties.csv'

In [ ]:
import pandas as pd
df = pd.read_csv(COUNTIES)

In [ ]:
df.head()

In [ ]:
df.info()

**2.** Przygotowanie danych do algorytmu

In [ ]:
# segmentacja
df.set_index('county', inplace=True)

In [ ]:
numerical_cols = ['rate', '< 95% CI', '> 95% CI', '< CI*rank', '> CI*rank', 'avg annual count', 'recent 5y trend']

In [ ]:
df = df[numerical_cols]

In [ ]:
# usuwanie nulli
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
df.var()

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_data = scaler.fit_transform(df)

In [ ]:
np.var(scaled_data)

1.0

In [ ]:
scaled_df = pd.DataFrame(data=scaled_data, columns=df.columns, index=df.index)
scaled_df.head()

**3.** Podział danych na 2 segmenty: label 0 i label 1

In [ ]:
kmeans = KMeans(n_clusters=2, n_init='auto', init='k-means++')

In [ ]:
kmeans

KMeans(n_clusters=2, n_init='auto')

In [ ]:
y_pred = kmeans.fit_predict(scaled_df)

In [ ]:
y_pred

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [ ]:
df['labels'] = y_pred
df.head()

In [ ]:
df.groupby('labels').std()

**4.** Zachowaj Series z ID counties jako indeksami i nazwami segmentów jako szeregiem danych. Nazwij szereg label-level-0 (atrybut .name)

In [ ]:
result_series = df['labels']
result_series.name = 'label-level-0'

result_series

**5.** Podział bazowej ramki na dwie ramki: counties_label_0 i counties_label_1

In [ ]:
counties_label_0 =  df[df["labels"] == 0]

In [ ]:
counties_label_1 = df[df["labels"] == 1]

**6.** Wytrenuj kolejne dwa razy algorytm K-means wykorzystując ramki counties_label_0 i counties_label_1. Ponownie podziel to na dwa segmenty. Dostaniesz w wyniku dwa szeregi, ustaw ich indeksy na indeksy obszarów. Nazwij te szeregi label-level-1. (Uwaga! Te dw szeregi to będą dwie różne zmienne na tym etapie).

**counties_label_0**

In [ ]:
# counties_label_0
scaled_data_0 = scaler.fit_transform(counties_label_0)
np.var(scaled_data_0)

scaled_df_0 = pd.DataFrame(data=scaled_data_0, columns=counties_label_0.columns, index=counties_label_0.index)
scaled_df_0.head()

In [ ]:
kmeans_0 = KMeans(n_clusters=2, n_init='auto', init='k-means++')
kmeans_0

KMeans(n_clusters=2, n_init='auto')

In [ ]:
y_pred_0 = kmeans_0.fit_predict(scaled_df_0)

In [ ]:
# miałam błąd: <ipython-input-88-f6a51a491be3>:1: SettingWithCopyWarning:
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead
# Pomogła dopiero kopia

counties_label_0_copy = counties_label_0.copy()

counties_label_0_copy.loc[:, 'labels'] = y_pred_0
counties_label_0_copy.head()

In [ ]:
result_series_0 = counties_label_0_copy['labels']
result_series_0.name = 'label-level-1'

result_series_0

**counties_label_1**

In [ ]:
# counties_label_1
scaled_data_1 = scaler.fit_transform(counties_label_1)
np.var(scaled_data_1)

scaled_df_1 = pd.DataFrame(data=scaled_data_1, columns=counties_label_1.columns, index=counties_label_1.index)
scaled_df_1.head()

In [ ]:
kmeans_1 = KMeans(n_clusters=2, n_init='auto', init='k-means++')
kmeans_1

KMeans(n_clusters=2, n_init='auto')

In [ ]:
y_pred_1 = kmeans_1.fit_predict(scaled_df_1)

In [ ]:
counties_label_1_copy = counties_label_1.copy()

counties_label_1_copy.loc[:, 'labels'] = y_pred_1
counties_label_1_copy.head()

In [ ]:
result_series_1 = counties_label_1_copy['labels']
result_series_1.name = 'label-level-1'

result_series_1

**7.** Złącz dwa szeregi metodą concat: https://pandas.pydata.org/docs/user_guide/merging.html (UWAGA! Chodzi o takie złączenie w którym wciąż mamy jedną kolumnę danych).

In [ ]:
frames = [result_series_0, result_series_1]

result_series_concat = pd.concat(frames)

any_values_equal_to_1 = (result_series_concat == 1).any()
any_values_equal_to_0 = (result_series_concat == 0).any()

In [ ]:
any_values_equal_to_1

True

In [ ]:
any_values_equal_to_0

True

**8.** Dołącz szereg z label-level-0 do bazowej ramki danych.
Dołącz połączony szereg z label-level-1 do bazowej ramki danych.
Zapisz nową ramkę do pliku counties_labeled.csv.

In [ ]:
result_series_df = pd.DataFrame(result_series)
result_series_df

In [ ]:
result_series_concat_df = pd.DataFrame(result_series_concat)
result_series_concat_df

In [ ]:
counties_labeled = df.join(result_series_df['label-level-0'])
counties_labeled_fin = counties_labeled.join(result_series_concat_df['label-level-1'])

counties_labeled_fin

,rate,< 95% CI,> 95% CI,< CI*rank,> CI*rank,avg annual count,recent 5y trend,labels,label-level-0,label-level-1
county,,,,,,,,,,
"Greer County, Oklahoma(6)",234.0,163.6,326.4,1.0,20.0,8.0,39.4,0,0,0
"Monona County, Iowa(7)",220.3,165.2,288.6,1.0,37.0,14.0,2.1,0,0,0
"Shelby County, Missouri(6)",214.1,149.9,297.6,1.0,69.0,9.0,4.0,0,0,0
"Pickens County, Alabama(6)",212.5,175.4,255.3,1.0,6.0,26.0,5.7,0,0,0
"Phelps County, Nebraska(6)",200.2,149.7,263.1,1.0,33.0,12.0,1.8,0,0,0
...,...,...,...,...,...,...,...,...,...,...
"Marion County, Georgia(7)",55.1,30.9,92.7,102.0,150.0,3.0,-4.4,1,1,0
"Bethel Census Area, Alaska(6)",53.5,31.7,84.7,13.0,15.0,4.0,-4.1,1,1,0
"Robertson County, Texas(6)",52.7,34.2,77.9,137.0,197.0,6.0,-15.7,1,1,0


In [ ]:
counties_labeled_fin = counties_labeled_fin.drop('labels', axis=1)

In [ ]:
RESULT = '/content/drive/MyDrive/DataScience/data/counties_labeled.csv'
counties_labeled_fin.to_csv(RESULT)

In [ ]:
counties_labeled_fin

,rate,< 95% CI,> 95% CI,< CI*rank,> CI*rank,avg annual count,recent 5y trend,label-level-0,label-level-1
county,,,,,,,,,
"Greer County, Oklahoma(6)",234.0,163.6,326.4,1.0,20.0,8.0,39.4,0,0
"Monona County, Iowa(7)",220.3,165.2,288.6,1.0,37.0,14.0,2.1,0,0
"Shelby County, Missouri(6)",214.1,149.9,297.6,1.0,69.0,9.0,4.0,0,0
"Pickens County, Alabama(6)",212.5,175.4,255.3,1.0,6.0,26.0,5.7,0,0
"Phelps County, Nebraska(6)",200.2,149.7,263.1,1.0,33.0,12.0,1.8,0,0
...,...,...,...,...,...,...,...,...,...
"Marion County, Georgia(7)",55.1,30.9,92.7,102.0,150.0,3.0,-4.4,1,0
"Bethel Census Area, Alaska(6)",53.5,31.7,84.7,13.0,15.0,4.0,-4.1,1,0
"Robertson County, Texas(6)",52.7,34.2,77.9,137.0,197.0,6.0,-15.7,1,0


**9.** Porównaj ze sobą poszczególne grupy:

+ grupę 0 pierwszego poziomu z grupą 1 pierwszego poziomu,

+ grupę 0 drugiego poziomu otrzymaną z danych grupy 0 pierwszego poziomu, z grupą 1 drugiego poizomu otrzymaną z danych z grupy 0 pierwszego poziomu.

Porównanie:

użyj metod statystyki (analiza średniej, wariancji, liczebności grup),
przeglądnij dane "ręcznie" i zastanów się co różni te grupy, wypisz 2-3 wnioski na poszczególny podział.

In [ ]:
grouped_1 = counties_labeled_fin.groupby('label-level-0')

# Analiza statystyczna
mean_values_1 = grouped_1[numerical_cols].mean()
variance_values_1 = grouped_1[numerical_cols].var()
count_values_1 = grouped_1[numerical_cols].count()

In [ ]:
mean_values_1

In [ ]:
variance_values_1

In [ ]:
count_values_1

In [ ]:
grouped_2 = counties_labeled_fin.groupby(['label-level-0','label-level-1'])

# Analiza statystyczna
mean_values_2 = grouped_2[numerical_cols].mean()
variance_values_2 = grouped_2[numerical_cols].var()
count_values_2 = grouped_2[numerical_cols].count()

In [ ]:
mean_values_1

,rate,< 95% CI,> 95% CI,< CI*rank,> CI*rank,avg annual count,recent 5y trend
label-level-0,,,,,,,
0,131.933566,110.732867,157.717546,3.998093,63.259377,126.144946,0.956771
1,101.244705,78.139817,130.915479,15.208758,98.941955,34.399185,-0.954786


**Wniosek 1:** Grupa 0 charakteryzuje się wyższymi średnimi wartościami dla większości analizowanych parametrów w porównaniu do Grupy 1. Istnieją jednak istotne różnice między grupami, które mogą być ważne dla dalszej analizy. Szczególnie warto zwrócić uwagę na zmienne < CI*rank i > CI*rank, gdzie Grupa 1 ma wyższe średnie wartości.

**Wniosek 2:** Grupa 0 ma średni trend wzrastający dla ostatnich 5 lat, a grupa 1 ma trend spadkowy, mogło to wpłynąć na podział na te dwie grupy

In [ ]:
mean_values_2

rate    < 95% CI    > 95% CI  < CI*rank  \
label-level-0 label-level-1                                                  
0             0              134.794216  103.971722  173.679692   1.638817   
              1              129.134088  117.349434  142.096730   6.306918   
1             0               83.240708   61.539381  112.012389  32.601770   
              1              106.626852   83.102381  136.566402  10.009259   

                              > CI*rank  avg annual count  recent 5y trend  
label-level-0 label-level-1                                                 
0             0               78.952442         20.043702         1.774679  
              1               47.901887        229.977358         0.156352  
1             0              109.150442         21.092920        -2.217257  
              1               95.890212         38.376984        -0.577381

**Wniosek 3:** Grupy 0-1 i 0-0 różnią się średnim trendem -> 0-0 jest bardziej wzrostowy niż 0-1.

**Wniosek 4:** znacza różnica występuje też w średnim avg annual count -> znacznie wyższy jest w grupie 0-1

In [ ]:
variance_values_1

,rate,< 95% CI,> 95% CI,< CI*rank,> CI*rank,avg annual count,recent 5y trend
label-level-0,,,,,,,
0,171.907664,231.650847,637.715048,23.994271,1157.944131,94022.088392,8.731311
1,163.521985,255.629351,299.753603,362.999189,2260.325883,11351.483710,6.357485


**Wniosek 5:** Wariancje różnych kolumn wskazują na zróżnicowany stopień rozproszenia danych w obu grupach. Duże wartości wariancji wskazują na większą zmienność, podczas gdy mniejsze wartości sugerują, że dane są bardziej skoncentrowane wokół średniej.

In [ ]:
variance_values_2

rate    < 95% CI    > 95% CI   < CI*rank  \
label-level-0 label-level-1                                                   
0             0              235.622425  225.411658  685.849832    1.801168   
              1               93.908219  149.422201   97.441022   34.951023   
1             0              106.367491  187.424798  162.409801  845.991819   
              1               54.736987  169.144286  202.144274  101.916470   

                               > CI*rank  avg annual count  recent 5y trend  
label-level-0 label-level-1                                                  
0             0              1087.327208        190.865527        14.439165  
              1               751.048296     164137.374802         1.859692  
1             0              3382.937266       1793.275772        12.965612  
              1              1888.246209      14146.147761         3.776878

**Wniosek 6:** Wariancja dla grupy 0-1 dla avg annual count i < CI * rank jest znacznie wyższa niż dla grupy 0-0, pozostałe wariancje są wyższe dla 0-0. Trend dla 0-1 jest bardzo mało rozproszony.

In [ ]:
count_values_1

,rate,< 95% CI,> 95% CI,< CI*rank,> CI*rank,avg annual count,recent 5y trend
label-level-0,,,,,,,
0,1573,1573,1573,1573,1573,1573,1573
1,982,982,982,982,982,982,982


**Wniosek 7:** Grupa 0 jest około 1.6 raza liczniejsza niż grupa 1.

In [ ]:
count_values_2

rate  < 95% CI  > 95% CI  < CI*rank  > CI*rank  \
label-level-0 label-level-1                                                   
0             0               778       778       778        778        778   
              1               795       795       795        795        795   
1             0               226       226       226        226        226   
              1               756       756       756        756        756   

                             avg annual count  recent 5y trend  
label-level-0 label-level-1                                     
0             0                           778              778  
              1                           795              795  
1             0                           226              226  
              1                           756              756

**Wniosek 8:** Grupa 0-1 i 0-0 mają podobną liczebność.

**END**